# Conversation Summary Node Test Notebook

This notebook tests the `conversation_summary.py` module functionality.

In [1]:

from dotenv import load_dotenv, find_dotenv

# .env 파일 로드
dotenv_path = find_dotenv()
load_dotenv(dotenv_path, override=True)


True

In [2]:
# Import required modules
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from app.agents.nodes.conversation_summary import summarize_conversation

/Users/kimjunghyeon/Desktop/workspace/ai-agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Test Setup

Create mock conversation data for testing

In [3]:
# Test Case 1: Short conversation (should not summarize)
def create_short_conversation():
    return {
        "messages": [
            HumanMessage(content="안녕하세요", id="msg1"),
            AIMessage(content="안녕하세요! 무엇을 도와드릴까요?", id="msg2")
        ]
    }


# Test Case 2: Long conversation (should summarize)
def create_long_conversation():
    return {
        "messages": [
            HumanMessage(content="파이썬 프로그래밍에 대해 알려주세요", id="msg1"),
            AIMessage(content="파이썬은 간단하고 읽기 쉬운 문법을 가진 프로그래밍 언어입니다.", id="msg2"),
            HumanMessage(content="파이썬으로 웹 개발이 가능한가요?", id="msg3"),
            AIMessage(content="네, Django나 FastAPI 같은 프레임워크로 웹 개발이 가능합니다.", id="msg4"),
            HumanMessage(content="FastAPI의 장점은 무엇인가요?", id="msg5"),
            AIMessage(content="FastAPI는 고성능이고 자동 API 문서화를 지원합니다.", id="msg6")
        ]
    }


# Test Case 3: Conversation with existing summary
def create_conversation_with_summary():
    return {
        "messages": [
            SystemMessage(content="[대화 요약]\n이전에 파이썬과 웹 개발에 대해 논의했습니다.", id="summary1"),
            HumanMessage(content="머신러닝도 파이썬으로 할 수 있나요?", id="msg7"),
            AIMessage(content="네, scikit-learn, TensorFlow, PyTorch 등을 사용할 수 있습니다.", id="msg8"),
            HumanMessage(content="추천하는 라이브러리는 무엇인가요?", id="msg9"),
            AIMessage(content="초보자에게는 scikit-learn을 추천합니다.", id="msg10")
        ],
        "summary": "이전에 파이썬과 웹 개발에 대해 논의했습니다."
    }


print("Test data created successfully!")

Test data created successfully!


## Test Case 1: Short Conversation

Test that short conversations (≤2 messages) are not summarized

In [4]:
async def test_short_conversation():
    print("=== Test Case 1: Short Conversation ===")

    state = create_short_conversation()
    print(f"Input messages: {len(state['messages'])} messages")
    for i, msg in enumerate(state['messages']):
        print(f"  {i + 1}: [{type(msg).__name__}] {msg.content}")

    result = await summarize_conversation(state)
    print(f"\nResult: {result}")
    print(f"Should return empty dict for short conversations: {result == {} }")


# Run the test
await test_short_conversation()

=== Test Case 1: Short Conversation ===
Input messages: 2 messages
  1: [HumanMessage] 안녕하세요
  2: [AIMessage] 안녕하세요! 무엇을 도와드릴까요?

Result: {}
Should return empty dict for short conversations: True


## Test Case 2: Long Conversation

Test that long conversations are properly summarized

In [7]:
async def test_long_conversation():
    print("=== Test Case 2: Long Conversation ===")

    state = create_long_conversation()
    print(f"Input messages: {len(state['messages'])} messages")
    for i, msg in enumerate(state['messages']):
        print(f"  {i + 1}: [{type(msg).__name__}] {msg.content[:50]}...")

    result = await summarize_conversation(state)

    print(f"\nResult keys: {list(result.keys())}")
    print(f"Summary: {result.get('summary', 'No summary')}")
    print(f"Messages to remove/add: {len(result.get('messages', []))}")

    messages = result.get('messages', [])

    for i, msg in enumerate(messages):
        class_name = type(msg).__name__
        if class_name == 'RemoveMessage':
            print(f"  {i + 1}: [RemoveMessage] id={msg.id}")
        else:
            print(f"  {i + 1}: [{class_name}] {msg.content}")

    # Check if RemoveMessage operations are correct
    messages = result.get('messages', [])
    remove_count = sum(1 for msg in messages if hasattr(msg, 'id') and msg.__class__.__name__ == 'RemoveMessage')
    system_count = sum(1 for msg in messages if isinstance(msg, SystemMessage))

    print(f"Remove operations: {remove_count}")
    print(f"Summary messages added: {system_count}")

# Run the test
await test_long_conversation()

=== Test Case 2: Long Conversation ===
Input messages: 6 messages
  1: [HumanMessage] 파이썬 프로그래밍에 대해 알려주세요...
  2: [AIMessage] 파이썬은 간단하고 읽기 쉬운 문법을 가진 프로그래밍 언어입니다....
  3: [HumanMessage] 파이썬으로 웹 개발이 가능한가요?...
  4: [AIMessage] 네, Django나 FastAPI 같은 프레임워크로 웹 개발이 가능합니다....
  5: [HumanMessage] FastAPI의 장점은 무엇인가요?...
  6: [AIMessage] FastAPI는 고성능이고 자동 API 문서화를 지원합니다....

Result keys: ['messages', 'summary']
Summary: 사용자가 파이썬 프로그래밍에 대해 질문하였고, AI는 파이썬이 간단하고 읽기 쉬운 문법을 가진 언어라고 설명했습니다. 이어서 사용자가 파이썬으로 웹 개발이 가능한지 물었고, AI는 Django나 FastAPI 같은 프레임워크를 통해 웹 개발이 가능하다고 답변했습니다.
Messages to remove/add: 5
  1: [RemoveMessage] id=msg1
  2: [RemoveMessage] id=msg2
  3: [RemoveMessage] id=msg3
  4: [RemoveMessage] id=msg4
  5: [SystemMessage] [대화 요약]
사용자가 파이썬 프로그래밍에 대해 질문하였고, AI는 파이썬이 간단하고 읽기 쉬운 문법을 가진 언어라고 설명했습니다. 이어서 사용자가 파이썬으로 웹 개발이 가능한지 물었고, AI는 Django나 FastAPI 같은 프레임워크를 통해 웹 개발이 가능하다고 답변했습니다.
Remove operations: 4
Summary messages added: 1


## Test Case 3: Conversation with Existing Summary

Test updating an existing summary

In [8]:
async def test_conversation_with_summary():
    print("=== Test Case 3: Conversation with Existing Summary ===")

    state = create_conversation_with_summary()
    print(f"Input messages: {len(state['messages'])} messages")
    print(f"Existing summary: {state.get('summary', 'None')}")

    for i, msg in enumerate(state['messages']):
        print(f"  {i + 1}: [{type(msg).__name__}] {msg.content[:50]}...")

    result = await summarize_conversation(state)

    print(f"\nResult keys: {list(result.keys())}")
    print(f"New summary: {result.get('summary', 'No summary')}")
    print(f"Messages operations: {len(result.get('messages', []))}")

    # Show the new summary message
    messages = result.get('messages', [])
    for msg in messages:
        if isinstance(msg, SystemMessage):
            print(f"New summary message: {msg.content}")


# Run the test
await test_conversation_with_summary()

=== Test Case 3: Conversation with Existing Summary ===
Input messages: 5 messages
Existing summary: 이전에 파이썬과 웹 개발에 대해 논의했습니다.
  1: [SystemMessage] [대화 요약]
이전에 파이썬과 웹 개발에 대해 논의했습니다....
  2: [HumanMessage] 머신러닝도 파이썬으로 할 수 있나요?...
  3: [AIMessage] 네, scikit-learn, TensorFlow, PyTorch 등을 사용할 수 있습니다...
  4: [HumanMessage] 추천하는 라이브러리는 무엇인가요?...
  5: [AIMessage] 초보자에게는 scikit-learn을 추천합니다....

Result keys: ['messages', 'summary']
New summary: 이전에 파이썬과 웹 개발에 대해 논의한 후, 사용자가 머신러닝을 파이썬으로 할 수 있는지 질문했습니다. AI는 scikit-learn, TensorFlow, PyTorch와 같은 라이브러리를 사용하여 머신러닝을 구현할 수 있다고 답변했습니다.
Messages operations: 4
New summary message: [대화 요약]
이전에 파이썬과 웹 개발에 대해 논의한 후, 사용자가 머신러닝을 파이썬으로 할 수 있는지 질문했습니다. AI는 scikit-learn, TensorFlow, PyTorch와 같은 라이브러리를 사용하여 머신러닝을 구현할 수 있다고 답변했습니다.
